In [2]:
from keras.models import  Model, load_model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import  ResNet50
from keras.optimizers import Adam
from keras.utils import  to_categorical
from keras.layers import Dense
from math import ceil
from keras import backend as k
from PIL import Image as im
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
import os
import cv2

## Build the model with 16 classes

In [35]:
base_model = VGG16(input_shape=(224,224,3),weights="imagenet")
base_model.layers.pop()
fc2_output = base_model.layers[-1].output
prediction = Dense(1000, activation="softmax", name="prediction")(fc2_output)
model = Model(inputs= base_model.input,outputs=prediction)

In [49]:
from keras.layers import Input,Conv2D,MaxPool2D
from keras.layers import Flatten,Dense,BatchNormalization
from keras.models import Model,load_model

In [76]:
def Vgg16():
    inputs = Input(shape=(224,224,3),name="Input")
    # block1
    x = Conv2D(64,(3,3),padding="same",activation="relu",name="block1_conv1")(inputs)
    x = Conv2D(64,(3,3),padding="same",activation="relu",name="block1_conv2")(x)
    x = MaxPool2D((2,2),strides=(2,2),padding="same",name="block1_pool")(x)
    #block2
    x = Conv2D(128,(3,3),padding="same",activation="relu",name="block2_conv1")(x)
    x = Conv2D(128,(3,3),padding="same",activation="relu",name="block2_conv2")(x)
    x = MaxPool2D((2,2),strides=(2,2),padding="same",name="block2_pool")(x)
    #block3
    x = Conv2D(256,(3,3),padding="same",activation="relu",name="block3_conv1")(x)
    x = Conv2D(256,(3,3),padding="same",activation="relu",name="block3_conv2")(x)
    x = Conv2D(256,(3,3),padding="same",activation="relu",name="block3_conv3")(x)
    x = MaxPool2D((2,2),strides=(2,2),padding="same",name="block3_pool")(x)
    #block4
    x = Conv2D(512,(3,3),padding="same",activation="relu",name="block4_conv1")(x)
    x = Conv2D(512,(3,3),padding="same",activation="relu",name="block4_conv2")(x)
    x = Conv2D(512,(3,3),padding="same",activation="relu",name="block4_conv3")(x)
    x = MaxPool2D((2,2),strides=(2,2),padding="same",name="block4_pool")(x)
    #block5
    x = Conv2D(512,(3,3),padding="same",activation="relu",name="block5_conv1")(x)
    x = Conv2D(512,(3,3),padding="same",activation="relu",name="block5_conv2")(x)
    x = Conv2D(512,(3,3),padding="same",activation="relu",name="block5_conv3")(x)
    x = MaxPool2D((2,2),strides=(2,2),padding="same",name="block5_pool")(x)
#     x = BatchNormalization(axis=3)(x)
    #block6
    x = Flatten(name="flatten")(x)
    x = Dense(4096,activation="relu",name="fc1")(x)
    x = Dense(4096,activation="relu",name="fc2")(x)
    predictions = Dense(16,activation="softmax",name="prediction")(x)
    model = Model(inputs=inputs,outputs=predictions)
    return model

In [82]:
k.clear_session()

In [83]:
model_vgg = Vgg16()

In [84]:
for layer in model_vgg.layers:
    layer.trainable = False

In [85]:
model_vgg.load_weights("/root/.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels.h5",by_name=True)

In [1]:
%run keras_verhicle_classfier/keras_vgg16.py --path  "keras_verhicle_classfier/1003_1.jpg"

Using TensorFlow backend.


InternalError: Failed to create session.

In [17]:
for layer in base_model.layers:
    layer.trainable = False

In [18]:
def get_data(path):
    img_list = []
    label_list = []
    for i in os.listdir(path):
        label =int(i.split("_")[1].split(".")[0])
        img_list.append(cv2.imread(os.path.join(path,i)))
        label_list.append(label)
    return img_list,label_list

In [11]:
train_path = "same_size"
img_list,label_list = get_data(train_path)

In [12]:
imgs = np.array([i for i in img_list])
lb = LabelBinarizer().fit(label_list)
labels = lb.transform(label_list)
train_imgs,valid_imgs,train_labels,valid_labels = train_test_split(imgs,labels,test_size=0.20,random_state=42)

In [86]:
adam = Adam(lr=0.01)
model_vgg.compile(optimizer=adam, loss="categorical_crossentropy",metrics=["accuracy"])

In [87]:
from keras.callbacks import LearningRateScheduler,EarlyStopping,ModelCheckpoint
batch_size = 32
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 **x)
earlystop = EarlyStopping(patience=10)
model_save = ModelCheckpoint(
                filepath="Vgg16_batch.h5", save_best_only=True, verbose=1
)
model_vgg.fit(train_imgs,train_labels,batch_size=batch_size,
          epochs=20, validation_data=(valid_imgs,valid_labels),
          callbacks=[annealer,earlystop,model_save]
         )

Train on 9974 samples, validate on 2494 samples
Epoch 1/20
9952/9974 [============================>.] - ETA: 0s - loss: 4.1620 - acc: 0.0622
Epoch 00001: val_loss improved from inf to 4.18852, saving model to Vgg16_batch.h5
9974/9974 [==============================] - 41s 4ms/step - loss: 4.1636 - acc: 0.0621 - val_loss: 4.1885 - val_acc: 0.0573
Epoch 2/20
9952/9974 [============================>.] - ETA: 0s - loss: 4.1627 - acc: 0.0621
Epoch 00002: val_loss did not improve
9974/9974 [==============================] - 37s 4ms/step - loss: 4.1636 - acc: 0.0621 - val_loss: 4.1885 - val_acc: 0.0573
Epoch 3/20
9952/9974 [============================>.] - ETA: 0s - loss: 4.1627 - acc: 0.0621
Epoch 00003: val_loss did not improve
9974/9974 [==============================] - 37s 4ms/step - loss: 4.1636 - acc: 0.0621 - val_loss: 4.1885 - val_acc: 0.0573
Epoch 4/20
9952/9974 [============================>.] - ETA: 0s - loss: 4.1644 - acc: 0.0621
Epoch 00004: val_loss did not improve
9974/9974 [

## Load the model and predict

In [2]:
model = load_model("vgg16.h5")

In [37]:
# model = load_model("data_avgg16.h5")
test_img ,test_label = get_data("test")
test_imgs = np.array(test_img)
predictions = model.predict(test_imgs)
predict_idxs = lb.inverse_transform(predictions)

In [38]:
predict_idxs

array([ 5,  1, 15,  6,  0,  4,  1,  1, 11,  8,  5,  5,  9, 15, 14, 15,  1,
        6,  3,  3,  5,  3,  9,  3,  1,  1,  4, 10, 11,  4,  9,  7,  4,  7,
        2, 14,  2,  3,  5,  7, 10,  0,  5,  6,  2,  9,  8,  2,  4,  3,  7,
        2,  1,  8,  9,  8,  5, 14,  4,  4,  7,  5,  5,  2, 10,  8,  1,  5,
        3,  7,  3,  2,  2,  2,  6, 14,  1,  2, 12,  5,  7,  1,  6,  7,  4,
        2,  8, 14,  5,  2,  6,  8,  6,  1, 10, 11,  3,  8,  5, 10,  5, 10,
        9,  7, 14, 13, 14, 15, 14,  2,  6,  8,  1,  5,  1, 12,  6,  5,  1,
        5,  0,  5,  5,  7, 10,  1,  9, 14,  4,  0,  1,  0,  9,  1,  5,  2,
       10,  0,  3,  2, 11,  7,  2,  5,  5,  4,  3,  9, 13,  6,  2,  1, 10,
       10,  8, 10, 13,  2,  1, 11, 11,  3, 11, 14, 11,  0,  2,  0, 10,  3,
        8, 14,  8, 11,  5,  5,  4,  6,  3,  5,  6,  2, 10,  8, 14,  0, 14,
        5,  4,  3, 15,  5,  2,  7, 10, 13,  6,  1, 10,  5])

In [4]:
def get_labelist(label_file):
    label_list = []
    fr = open(label_file,"rb")
    for line in fr:
        label = line.strip().split(" ")[1]
        label_list.append(label)
    return label_list

In [5]:
label_list = get_labelist("label.txt")
print(label_list[0])

大众帕萨特1


In [40]:
from __future__ import  print_function
true_nums = 0
for i in range(len(test_label)):
    if test_label[i] == list(predict_idxs)[i]:
        true_nums += 1
print("Test accuracy: {}%".format(float(true_nums) * 100/len(test_label)))

Test accuracy: 93.5%


In [59]:
for layer in model.layers:
    try:
        print(list(layer.get_weights()[0])[0].shape)
    except:
        pass

(3, 3, 64)
(3, 64, 64)
(3, 64, 128)
(3, 128, 128)
(3, 128, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 512)
(3, 512, 512)
(3, 512, 512)
(3, 512, 512)
(3, 512, 512)
(3, 512, 512)
(4096,)
(4096,)
(276,)


## Data augmentation and train

In [98]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization = False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False)
datagen.fit(train_imgs)

In [123]:
model.fit_generator(datagen.flow(train_imgs,train_labels,batch_size=32),
                    epochs=50,validation_data=(valid_imgs,valid_labels),workers=4)

Epoch 1/50
315/315 [==============================] - 37s 116ms/step - loss: 0.2869 - acc: 0.9225 - val_loss: 0.2735 - val_acc: 0.9467
Epoch 2/50
315/315 [==============================] - 36s 116ms/step - loss: 0.2205 - acc: 0.9409 - val_loss: 0.2597 - val_acc: 0.9463
Epoch 3/50
315/315 [==============================] - 37s 116ms/step - loss: 0.1914 - acc: 0.9463 - val_loss: 0.2771 - val_acc: 0.9431
Epoch 4/50
315/315 [==============================] - 37s 117ms/step - loss: 0.1656 - acc: 0.9516 - val_loss: 0.2710 - val_acc: 0.9483
Epoch 5/50
315/315 [==============================] - 37s 117ms/step - loss: 0.1426 - acc: 0.9573 - val_loss: 0.2568 - val_acc: 0.9479
Epoch 6/50
315/315 [==============================] - 37s 116ms/step - loss: 0.1443 - acc: 0.9579 - val_loss: 0.2584 - val_acc: 0.9487
Epoch 7/50
315/315 [==============================] - 37s 117ms/step - loss: 0.1360 - acc: 0.9595 - val_loss: 0.2838 - val_acc: 0.9507
Epoch 8/50
315/315 [==============================] - 3

In [124]:
model.save("data_avgg16.h5")